In [5]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [6]:
adeft_grounder = AdeftGrounder()

In [7]:
shortforms = ['GH']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [8]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [9]:
list(zip(longforms, counts))

[('growth hormone', 6773),
 ('gestational hypertension', 49),
 ('glycoside hydrolase', 42),
 ('general health', 14),
 ('glycogenic hepatopathy', 13),
 ('growth hor mone', 11),
 ('garlic homogenate', 10),
 ('gelatin hydrogel', 9),
 ('genetic hypertension', 9),
 ('gingival hyperplasia', 8),
 ('glycosyl hydrolase', 8),
 ('gelam honey', 7),
 ('glucosamine hydrochloride', 7),
 ('gestational hypothyroid', 6),
 ('group housed', 6),
 ('glucosyl hesperidin', 5),
 ('graphene hydrogel', 5),
 ('glenohumeral', 5),
 ('geniohyoid', 5),
 ('genital hiatus', 4),
 ('gastric heterotopia', 4),
 ('genetic hemochromatosis', 4),
 ('grass hay', 4),
 ('graves hyperthyroidism', 4),
 ('guest host', 4),
 ('growth', 4),
 ('general health perceptions', 4),
 ('glycan hole', 3),
 ('glycine hydroxamate', 3),
 ('grain hardness', 3),
 ('global health', 3),
 ('gelatin − hydroxyphenyl propionic acid', 3)]

In [10]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [11]:
names

{'CHEBI:CHEBI:37845': 'growth hormone',
 'MESH:D006262': 'Health',
 'MESH:D008107': 'Liver Diseases',
 'MESH:D020100': 'Hydrogels',
 'MESH:D005885': 'Gingival Hyperplasia',
 'MESH:D006722': 'Honey',
 'CHEBI:CHEBI:36807': 'hydrochloride',
 'MESH:D007037': 'Hypothyroidism',
 'MESH:C487382': 'glucosyl hesperidin',
 'MESH:D002828': 'Choristoma',
 'MESH:D006432': 'Hemochromatosis',
 'EFO:0000532': 'host',
 'MESH:D006128': 'Growth',
 'MESH:D010465': 'Perception',
 'HGNC:20511': 'TMEM121',
 'CHEBI:CHEBI:75426': 'glycine hydroxamate',
 'MESH:D006244': 'Hardness',
 'MESH:D014943': 'Global Health',
 'CHEBI:CHEBI:32980': 'phloretic acid',
 'HGNC:4261': 'GH1',
 'MESH:D006026': 'Glycoside Hydrolases'}

In [12]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-02 23:23:15] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [13]:
result = [grounding_map, names, pos_labels]

In [14]:
result

[{'garlic homogenate': 'ungrounded',
  'gastric heterotopia': 'MESH:D002828',
  'gelam honey': 'MESH:D006722',
  'gelatin hydrogel': 'ungrounded',
  'gelatin − hydroxyphenyl propionic acid': 'ungrounded',
  'general health': 'MESH:D006262',
  'general health perceptions': 'ungrounded',
  'genetic hemochromatosis': 'MESH:D006432',
  'genetic hypertension': 'ungrounded',
  'geniohyoid': 'NCIT:C52930',
  'genital hiatus': 'ungrounded',
  'gestational hypertension': 'ungrounded',
  'gestational hypothyroid': 'MESH:D007037',
  'gingival hyperplasia': 'MESH:D005885',
  'glenohumeral': 'ungrounded',
  'global health': 'MESH:D014943',
  'glucosamine hydrochloride': 'NCIT:C83732',
  'glucosyl hesperidin': 'MESH:C487382',
  'glycan hole': 'ungrounded',
  'glycine hydroxamate': 'CHEBI:CHEBI:75426',
  'glycogenic hepatopathy': 'ungrounded',
  'glycoside hydrolase': 'MESH:D006026',
  'glycosyl hydrolase': 'MESH:D006026',
  'grain hardness': 'ungrounded',
  'graphene hydrogel': 'ungrounded',
  'gras

In [15]:
grounding_map, names, pos_labels = [{'garlic homogenate': 'ungrounded',
  'gastric heterotopia': 'MESH:D002828',
  'gelam honey': 'MESH:D006722',
  'gelatin hydrogel': 'ungrounded',
  'gelatin − hydroxyphenyl propionic acid': 'ungrounded',
  'general health': 'MESH:D006262',
  'general health perceptions': 'ungrounded',
  'genetic hemochromatosis': 'MESH:D006432',
  'genetic hypertension': 'ungrounded',
  'geniohyoid': 'NCIT:C52930',
  'genital hiatus': 'ungrounded',
  'gestational hypertension': 'ungrounded',
  'gestational hypothyroid': 'MESH:D007037',
  'gingival hyperplasia': 'MESH:D005885',
  'glenohumeral': 'ungrounded',
  'global health': 'MESH:D014943',
  'glucosamine hydrochloride': 'NCIT:C83732',
  'glucosyl hesperidin': 'MESH:C487382',
  'glycan hole': 'ungrounded',
  'glycine hydroxamate': 'CHEBI:CHEBI:75426',
  'glycogenic hepatopathy': 'ungrounded',
  'glycoside hydrolase': 'MESH:D006026',
  'glycosyl hydrolase': 'MESH:D006026',
  'grain hardness': 'ungrounded',
  'graphene hydrogel': 'ungrounded',
  'grass hay': 'ungrounded',
  'graves hyperthyroidism': 'NCIT:C3071',
  'group housed': 'ungrounded',
  'growth': 'ungrounded',
  'growth hor mone': 'HGNC:4261',
  'growth hormone': 'HGNC:4261',
  'guest host': 'ungrounded'},
 {'MESH:D002828': 'Choristoma',
  'MESH:D006722': 'Honey',
  'MESH:D006262': 'Health',
  'MESH:D006432': 'Hemochromatosis',
  'NCIT:C52930': 'Geniohyoid',
  'MESH:D007037': 'Hypothyroidism',
  'MESH:D005885': 'Gingival Hyperplasia',
  'MESH:D014943': 'Global Health',
  'NCIT:C83732': 'Glucosamine Hydrochloride',
  'MESH:C487382': 'glucosyl hesperidin',
  'CHEBI:CHEBI:75426': 'glycine hydroxamate',
  'MESH:D006026': 'Glycoside Hydrolases',
  'NCIT:C3071': 'Graves Disease',
  'HGNC:4261': 'GH1'},
 ['HGNC:4261', 'MESH:D006026']]

In [16]:
excluded_longforms = ['growth']

In [17]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [18]:
additional_entities = {}

In [19]:
unambiguous_agent_texts = {}

In [20]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [21]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-02 23:38:32] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-02 23:40:13] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9923180045053153 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [22]:
classifier.stats

{'label_distribution': {'HGNC:4261': 5491,
  'ungrounded': 86,
  'NCIT:C52930': 4,
  'CHEBI:CHEBI:75426': 3,
  'MESH:D006432': 4,
  'MESH:D005885': 8,
  'MESH:D006026': 32,
  'NCIT:C3071': 3,
  'MESH:D006262': 12,
  'MESH:D014943': 2,
  'MESH:D007037': 1,
  'MESH:D002828': 2,
  'MESH:C487382': 3,
  'MESH:D006722': 4,
  'NCIT:C83732': 2},
 'f1': {'mean': 0.992318, 'std': 0.001003},
 'precision': {'mean': 0.986607, 'std': 0.00278},
 'recall': {'mean': 0.998371, 'std': 0.001201},
 'NCIT:C3071': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'NCIT:C83732': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D002828': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D014943': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.

In [23]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [24]:
disamb.dump(model_name, results_path)

In [25]:
print(disamb.info())

Disambiguation model for GH

Produces the disambiguations:
	Choristoma	MESH:D002828
	GH1*	HGNC:4261
	Geniohyoid	NCIT:C52930
	Gingival Hyperplasia	MESH:D005885
	Global Health	MESH:D014943
	Glucosamine Hydrochloride	NCIT:C83732
	Glycoside Hydrolases*	MESH:D006026
	Graves Disease	NCIT:C3071
	Health	MESH:D006262
	Hemochromatosis	MESH:D006432
	Honey	MESH:D006722
	Hypothyroidism	MESH:D007037
	glucosyl hesperidin	MESH:C487382
	glycine hydroxamate	CHEBI:CHEBI:75426

Class level metrics:
--------------------
Grounding                	Count	F1     
                      GH1*	5491	0.99304
               Ungrounded	  86	0.68898
     Glycoside Hydrolases*	  32	0.87189
                   Health	  12	0.31333
     Gingival Hyperplasia	   8	    0.0
               Geniohyoid	   4	    0.0
          Hemochromatosis	   4	    0.4
                    Honey	   4	    0.0
      glycine hydroxamate	   3	    0.4
           Graves Disease	   3	    0.0
      glucosyl hesperidin	   3	    0.0
            Global Healt

In [26]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1